# PyCity Schools Analysis


In [ ]:
# Import important libraries
import pandas as pd
from pathlib import Path

In [ ]:
# Load files 
load_school_data = Path("Resources/schools_complete.csv")
load_student_data = Path("Resources/students_complete.csv")

In [ ]:
# Read files into dataframes using Pandas
school_data = pd.read_csv(load_school_data)
student_data = pd.read_csv(load_student_data)

In [ ]:
# Merge the datasets
complete_school_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
complete_school_data.head()

# District Summary

In [ ]:
# Calculate the total number of unique schools
school_count = complete_school_data['school_name'].nunique()
school_count

In [ ]:
# Calculate the total number of students
total_students = complete_school_data['Student ID'].count()
total_students

In [ ]:
# Calculate the total budget
total_budget = school_data['budget'].sum()
total_budget

In [ ]:
# Calculate the average math score
average_score_math = complete_school_data['math_score'].mean()
average_score_math

In [ ]:
# Calculate the avarage reading score
average_score_reading = complete_school_data['reading_score'].mean()
average_score_reading

In [ ]:
# Calculate the percentage of students who scored 70 or more in math
students_passed_math_count = complete_school_data[(complete_school_data["math_score"] >= 70)].count()["student_name"]
percentage_passed_math = students_passed_math_count / float(total_students) * 100
percentage_passed_math

In [ ]:
# Calculate the percentage of students who scored 70 or more in reading
students_passed_reading_count = complete_school_data[(complete_school_data["reading_score"] >= 70)].count()["student_name"]
percentage_passed_reading = students_passed_reading_count / float(total_students) * 100
percentage_passed_reading

In [ ]:
# Calculate the percentage of students who passed math and reading
passed_math_reading_count = complete_school_data[
    (complete_school_data["math_score"] >= 70) & (complete_school_data["reading_score"] >= 70)
].count()["student_name"]
total_passing_rate = passed_math_reading_count / float(total_students) * 100
total_passing_rate

In [ ]:
# Create a high-level snapshot of the district's key metrics in a dataframe
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                 "Total Students": [total_students],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [average_score_math],
                                 "Average Reading Score": [average_score_reading],
                                 "Percentage Passed Math": [percentage_passed_math],
                                 "Percentage Passed Reading": [percentage_passed_reading],
                                 "Total Passing Rate": [total_passing_rate]})

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

# School Summary

In [ ]:
# Select the type per school from school data
school_type = school_data.set_index(["school_name"])["type"]

In [ ]:
# Calculate the total student count per school form school_data
student_per_school_count = complete_school_data["school_name"].value_counts()

In [ ]:
# Calculate the total school budget and per capita spending per school from school_data
total_budget_per_school = complete_school_data.groupby("school_name")["budget"].mean()
per_school_capita = total_budget_per_school / student_per_school_count

In [ ]:
# Calculate the average test scores per school from complete_school_data
avg_math_score_per_school = complete_school_data.groupby("school_name")["math_score"].mean()
avg_reading_score_per_school = complete_school_data.groupby("school_name")["reading_score"].mean()

In [ ]:
# Calculate the number of students per school with math scores of 70 or higher from complete_school_data
students_passed_math = complete_school_data[(complete_school_data["math_score"] >= 70)]
school_students_passed_math = students_passed_math.groupby(["school_name"]).count()["student_name"] / student_per_school_count * 100

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher from complete_school_data
students_passed_reading = complete_school_data[(complete_school_data["reading_score"] >= 70)]
school_students_passed_reading  = students_passed_reading.groupby(["school_name"]).count()["student_name"] / student_per_school_count * 100

In [ ]:
# Calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passed_math_and_reading = complete_school_data[
    (complete_school_data["reading_score"] >= 70) & (complete_school_data["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passed_math_and_reading.groupby(["school_name"]).size()

In [ ]:
# Calculate the passing rates
per_school_passing_math = school_students_passed_math
per_school_passing_reading = school_students_passed_reading
overall_passing_rate = school_students_passing_math_and_reading / student_per_school_count * 100

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type": school_type,
                                   "Total Students": student_per_school_count,
                                   "Total School Budget": total_budget_per_school,
                                   "Budget Per Student": per_school_capita,
                                   "Average Math Score": avg_math_score_per_school,
                                   "Average Reading Score": avg_reading_score_per_school,
                                   "Percentage Passed Math":per_school_passing_math,
                                   "Percentage Passing Reading": per_school_passing_reading,
                                   "Percentage Overall Passing Rate": overall_passing_rate})

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Budget Per Student"] = per_school_summary["Budget Per Student"].map("${:,.2f}".format)

per_school_summary

# Highest-Performing Schools (by Percentage Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(["Percentage Overall Passing Rate"], ascending=False)
top_schools.head(5)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["Percentage Overall Passing Rate"], ascending=True)
bottom_schools.head(5)

In [ ]:
# Separate the data by grade
ninth_graders = complete_school_data[(complete_school_data["grade"] == "9th")]
tenth_graders = complete_school_data[(complete_school_data["grade"] == "10th")]
eleventh_graders = complete_school_data[(complete_school_data["grade"] == "11th")]
twelfth_graders = complete_school_data[(complete_school_data["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_graders_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_graders_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_graders_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_graders_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th": ninth_graders_math_scores,
                                     "10th": tenth_graders_math_scores,
                                     "11th": eleventh_graders_math_scores,
                                     "12th": twelfth_graders_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = complete_school_data[(complete_school_data["grade"] == "9th")]
tenth_graders = complete_school_data[(complete_school_data["grade"] == "10th")]
eleventh_graders = complete_school_data[(complete_school_data["grade"] == "11th")]
twelfth_graders = complete_school_data[(complete_school_data["grade"] == "12th")]

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_graders_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_graders_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_graders_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_graders_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th": ninth_graders_reading_scores,
                                     "10th": tenth_graders_reading_scores,
                                     "11th": eleventh_graders_reading_scores,
                                     "12th": twelfth_graders_reading_scores})

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

In [ ]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
titles = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=titles)
school_spending_df

In [ ]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Percentage Passed Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Percentage Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Percentage Overall Passing Rate"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
                                 "Average Reading Score": spending_reading_scores,
                                 "Percentage Passed Math": spending_passing_math,
                                 "Percentage Passing Reading": spending_passing_reading,
                                 "Percentage Overall Passing Rate": overall_passing_spending
                                 })
spending_summary

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorise the spending based on the bins
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=labels)
per_school_summary

In [ ]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary.groupby(["School Size"], observed=True)["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"], observed=True)["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"], observed=True)["Percentage Passed Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"], observed=True)["Percentage Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"], observed=True)["Percentage Overall Passing Rate"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "Percentage Passed Math": size_passing_math,
    "Percentage Passing Reading": size_passing_reading,
    "Percentage Overall Passing Rate": size_overall_passing
    })
size_summary

# Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["Percentage Passed Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["Percentage Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["Percentage Overall Passing Rate"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Average Math Score": average_math_score_by_type,
                             "Average Reading Score": average_reading_score_by_type,
                             "Percentage Passed Math": average_percent_passing_math_by_type,
                             "Percentage Passing Reading": average_percent_passing_reading_by_type,
                             "Percentage Overall Passing Rate": average_percent_overall_passing_by_type})

# Display results
type_summary